In [3]:
import numpy as np
import sys
module_path = '/home/diplomski-rad/consensus-net/src/python/dataset/'
if module_path not in sys.path:
    print('Adding dataset module.')
    sys.path.append(module_path)

import dataset

Adding dataset module.


## Create neighbourhood from reads-to-ref pileups for training

In [5]:
X_paths = ['/home/diplomski-rad/blade/pileups/e-coli-NCTC86-pysam-X-dataset.npy',
           '/home/diplomski-rad/blade/pileups/m-morgani-NCTC235-pysam-X-dataset.npy',
           '/home/diplomski-rad/blade/pileups/s-enterica-NCTC92-pysam-X-dataset.npy',
           '/home/diplomski-rad/blade/pileups/s-enterica-NXTC129-pysam-X-dataset.npy']
y_paths = ['/home/diplomski-rad/blade/pileups/e-coli-NCTC86-pysam-y-dataset.npy',
           '/home/diplomski-rad/blade/pileups/m-morgani-NCTC235-pysam-y-dataset.npy',
           '/home/diplomski-rad/blade/pileups/s-enterica-NCTC92-pysam-y-dataset.npy',
           '/home/diplomski-rad/blade/pileups/s-enterica-NXTC129-pysam-y-dataset.npy']
neighbourhood_size = 10

X, y = dataset.create_dataset_with_neighbourhood(X_paths, y_paths, neighbourhood_size)

Parsing  /home/diplomski-rad/blade/pileups/e-coli-NCTC86-pysam-X-dataset.npy  and  /home/diplomski-rad/blade/pileups/e-coli-NCTC86-pysam-y-dataset.npy


  0% (4186 of 4641652) |                 | Elapsed Time: 0:00:00 ETA:   0:03:41

Creating dataset with neighbrouhood ...


100% (4641652 of 4641652) |##############| Elapsed Time: 0:03:34 Time:  0:03:34


Parsing  /home/diplomski-rad/blade/pileups/m-morgani-NCTC235-pysam-X-dataset.npy  and  /home/diplomski-rad/blade/pileups/m-morgani-NCTC235-pysam-y-dataset.npy


...

100% (4809037 of 4809037) |##############| Elapsed Time: 0:03:30 Time:  0:03:30


Parsing  /home/diplomski-rad/blade/pileups/s-enterica-NXTC129-pysam-X-dataset.npy  and  /home/diplomski-rad/blade/pileups/s-enterica-NXTC129-pysam-y-dataset.npy


  0% (4203 of 4809037) |                 | Elapsed Time: 0:00:00 ETA:   0:03:48

Creating dataset with neighbrouhood ...


100% (4809037 of 4809037) |##############| Elapsed Time: 0:03:44 Time:  0:03:44


In [6]:
np.save('./pysam-all-dataset-n10-X.npy', X)
np.save('./pysam-all-dataset-n10-y.npy', y)

## Train model

In [7]:
X, y, X_train, X_validate, y_train, y_validate = dataset.read_dataset_and_reshape_for_conv(
    './pysam-all-dataset-n10-X.npy', './pysam-all-dataset-n10-y.npy', 0.1)

  0% (6091 of 16668390) |                | Elapsed Time: 0:00:00 ETA:   0:09:07

X shape before reshaping: (16668390, 21, 4)
y shape before reshaping: (16668390, 4)


100% (16668390 of 16668390) |############| Elapsed Time: 0:11:21 Time:  0:11:21


X shape after reshaping: (16668390, 21, 1, 4)
y shape after reshaping: (16668390, 4)
Splitting to train and validation set.
X_train shape: (15001551, 21, 1, 4)
X_validate shape: (1666839, 21, 1, 4)
y_train: (15001551, 4)
y_validate: (1666839, 4)


In [9]:
np.save('./pysam-all-dataset-n10-X-reshaped', X)
np.save('./pysam-all-dataset-n10-y-reshaped', y)
np.save('./pysam-all-dataset-n10-X-reshaped-train', X_train)
np.save('./pysam-all-dataset-n10-X-reshaped-validate', X_validate)
np.save('./pysam-all-dataset-n10-y-reshaped-train', y_train)
np.save('./pysam-all-dataset-n10-y-reshaped-validate', y_validate)

In [15]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPool2D

In [14]:
input_layer = Input(shape=(21, 1, 4))
conv_1 = Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(input_layer)
conv_2 = Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(conv_1)

flatten = Flatten()(conv_2)
predictions = Dense(4, activation='softmax')(flatten)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

batch_size = 10000
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 21, 1, 4)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 21, 1, 20)         740       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 1, 20)         3620      
_________________________________________________________________
flatten_5 (Flatten)          (None, 420)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 1684      
Total params: 6,044
Trainable params: 6,044
Non-trainable params: 0
_________________________________________________________________
None
Train on 15001551 samples, validate on 1666839 samples
Epoch 1/20
15001551/15001551 [==============================] - 58s 4us/step - l

KeyboardInterrupt: 

In [16]:
input_layer = Input(shape=(21, 1, 4))
conv_1 = Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(input_layer)
pool_1 = MaxPool2D(pool_size=(2, 1))(conv_1)
conv_2 = Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(pool_1)

flatten = Flatten()(conv_2)
predictions = Dense(4, activation='softmax')(flatten)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

batch_size = 10000
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 21, 1, 4)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 21, 1, 20)         740       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 1, 20)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 1, 20)         3620      
_________________________________________________________________
flatten_6 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 804       
Total params: 5,164
Trainable params: 5,164
Non-trainable params: 0
_________________________________________________________________
None
Tra

In [17]:
model.save('./model-conv_pool_conv.h5')

In [18]:
input_layer = Input(shape=(21, 1, 4))
conv_1 = Conv2D(filters=40, kernel_size=3, padding='same', activation='relu')(input_layer)
pool_1 = MaxPool2D(pool_size=(2, 1))(conv_1)
conv_2 = Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(pool_1)

flatten = Flatten()(conv_2)
predictions = Dense(4, activation='softmax')(flatten)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

batch_size = 10000
epochs = 20

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 21, 1, 4)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 21, 1, 40)         1480      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 1, 40)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 1, 20)         7220      
_________________________________________________________________
flatten_7 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 804       
Total params: 9,504
Trainable params: 9,504
Non-trainable params: 0
_________________________________________________________________
None
Tra

In [19]:
model.save('./model-conv40_pool_conv20.h5')

In [22]:
def lr_schedule(epoch, lr):
    if epoch % 5 == 0:
        return lr * 0.95
    return lr

from keras.callbacks import LearningRateScheduler

batch_size = 10000
epochs = 20

lr_callback = LearningRateScheduler(lr_schedule)
callbacks = [lr_callback]
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate), callbacks=callbacks)

Train on 15001551 samples, validate on 1666839 samples
Epoch 1/20
15001551/15001551 [==============================] - 53s 4us/step - loss: 0.0107 - acc: 0.9967 - val_loss: 0.0111 - val_acc: 0.9967
Epoch 2/20
15001551/15001551 [==============================] - 49s 3us/step - loss: 0.0106 - acc: 0.9968 - val_loss: 0.0107 - val_acc: 0.9968
Epoch 3/20
15001551/15001551 [==============================] - 48s 3us/step - loss: 0.0105 - acc: 0.9968 - val_loss: 0.0110 - val_acc: 0.9967
Epoch 4/20
15001551/15001551 [==============================] - 57s 4us/step - loss: 0.0104 - acc: 0.9969 - val_loss: 0.0107 - val_acc: 0.9968
Epoch 5/20
15001551/15001551 [==============================] - 61s 4us/step - loss: 0.0102 - acc: 0.9969 - val_loss: 0.0126 - val_acc: 0.9963
Epoch 6/20
15001551/15001551 [==============================] - 64s 4us/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.0100 - val_acc: 0.9970
Epoch 7/20
15001551/15001551 [==============================] - 62s 4us/step - loss: 0.

In [23]:
model.save('./model-conv40_pool_conv20.h5')